In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image
import tensorflow as tf

In [2]:
classes = np.array(["Bread", "Dairy product", "Dessert", "Egg", "Fried food",
	"Meat", "Noodles/Pasta", "Rice", "Seafood", "Soup",
	"Vegetable/Fruit"])

In [9]:
training_dir = "Food-11/training/"
validation_dir = "Food-11/validation/"
evaluation_dir = "Food-11/evaluation/"

In [3]:
training_dir = "Food-11/training/"
training_images = os.listdir(training_dir)
training_labels = np.array([int(f.split('_')[0]) for f in training_images ])

ValueError: invalid literal for int() with base 10: 'class'

In [4]:
validation_dir = "Food-11/validation/"
validation_images = os.listdir(validation_dir)
validation_labels = np.array([int(f.split('_')[0]) for f in validation_images ])

In [5]:
evaluation_dir = "Food-11/evaluation/"
evaluation_images = os.listdir(evaluation_dir)
evaluation_labels = np.array([int(f.split('_')[0]) for f in evaluation_images ])

In [7]:
%cd Food-11/training/

C:\Users\ASUS\Desktop\ECE6143\ECE-GY_6143_Fall_2023\project\Food-11\training


In [10]:
# loop over classes
for i in range(len(classes)):
  # make a directory for this class inside Food-11/training
  try:
    os.mkdir("class_%02d" % i)
  except FileExistsError:
    pass
  # get list of files inside Food-11/training that have this class label
  files = [f for f in os.listdir('C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/Food-11/training/') if f.startswith("%d_" % i)]
  # move each file to the subdirectory for the class
  for f in files:
    shutil.move(f, "class_%02d/" % i)

In [15]:
%cd C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/Food-11/validation/

C:\Users\ASUS\Desktop\ECE6143\ECE-GY_6143_Fall_2023\project\Food-11\validation


In [16]:

for i in range(len(classes)):
  try:
    os.mkdir("class_%02d" % i)
  except FileExistsError:
    pass
  files = [f for f in os.listdir('C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/Food-11/validation/') if f.startswith("%d_" % i)]
  for f in files:
    shutil.move(f, "class_%02d/" % i)

In [18]:
%cd C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/Food-11/evaluation/

for i in range(len(classes)):
  try:
    os.mkdir("class_%02d" % i)
  except FileExistsError:
    pass
  files = [f for f in os.listdir('C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/Food-11/evaluation/') if f.startswith("%d_" % i)]
  for f in files:
    shutil.move(f, "class_%02d/" % i)

C:\Users\ASUS\Desktop\ECE6143\ECE-GY_6143_Fall_2023\project\Food-11\evaluation


In [4]:
# go back to default working directory
%cd C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/

C:\Users\ASUS\Desktop\ECE6143\ECE-GY_6143_Fall_2023\project


In [5]:
# TODO 1 (optional) - design choice on batch size, image shape
BATCH_SIZE=32
INPUT_IMG_SIZE = 224

In [6]:
# TODO 2 (optional) - design choices on image transformations for data augmentation

# prepare ImageDataGenerator to create augmented training samples
training_aug = tf.keras.preprocessing.image.ImageDataGenerator(
  rescale=1.0/255,          # rescales so each pixel is in 0-1 range
	rotation_range=1,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.1,
	horizontal_flip=True,
	fill_mode="nearest")

# prepare generator that pulls images from directory (and resizes)
training_gen = training_aug.flow_from_directory(
	'C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/' + training_dir,
	target_size=(INPUT_IMG_SIZE, INPUT_IMG_SIZE),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE,
  class_mode='sparse')

Found 9866 images belonging to 11 classes.


In [7]:
training_gen.class_indices

{'class_00': 0,
 'class_01': 1,
 'class_02': 2,
 'class_03': 3,
 'class_04': 4,
 'class_05': 5,
 'class_06': 6,
 'class_07': 7,
 'class_08': 8,
 'class_09': 9,
 'class_10': 10}

In [10]:
validation_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255
)

validation_gen = validation_aug.flow_from_directory(
	'C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/' + validation_dir,
	target_size=(INPUT_IMG_SIZE, INPUT_IMG_SIZE),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE,
  class_mode='sparse')

Found 3430 images belonging to 11 classes.


In [11]:
evaluation_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255
)

evaluation_gen = evaluation_aug.flow_from_directory(
	'C:/Users/ASUS/Desktop/ECE6143/ECE-GY_6143_Fall_2023/project/' + evaluation_dir,
	target_size=(INPUT_IMG_SIZE, INPUT_IMG_SIZE),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE,
  class_mode='sparse')

Found 3347 images belonging to 11 classes.


In [12]:
base_model = tf.keras.applications.ResNet101V2(
  input_shape=(INPUT_IMG_SIZE,INPUT_IMG_SIZE,3),
  include_top=False,
  pooling='avg'
)

In [13]:
base_model.summary()

Model: "resnet101v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                        

In [14]:
# TODO 4 (required) - prepare model with new classification head
num_classes = len(classes)
# freeze base model
base_model.trainable = False

model = tf.keras.models.Sequential()
# add to your model here...
model.add(base_model)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 2048)              42626560  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 11)                22539     
                                                                 
Total params: 42,649,099
Trainable params: 22,539
Non-trainable params: 42,626,560
_________________________________________________________________


In [16]:
# TODO 5 (required) - design training parameters, and compile

# model.compile(...)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

In [23]:
tf.config.list_physical_devices("GPU")

[]

In [18]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


Please install GPU version of TF


In [35]:
# TODO 6 (required) - fit model, you decide how many epochs
# note: you can get the number of training samples with training_gen.n
# and the number of validation samples with validation_gen.n

# n_epochs = ...
# hist = model.fit(...)
tf.debugging.set_log_device_placement(True)

# 设置早停
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,  # 持续10个epochs没有改善则停止
    restore_best_weights=True  # 恢复最佳模型权重
)

# 设定训练周期数
n_epochs = 20  # 这是一个起始值，可能需要根据实际情况调整

# 训练模型
hist = model.fit(
    training_gen,
    steps_per_epoch=training_gen.n // BATCH_SIZE,
    epochs=n_epochs,
    validation_data=validation_gen,
    validation_steps=validation_gen.n // BATCH_SIZE,
    callbacks=[early_stopping]
)

Epoch 1/20
  8/308 [..............................] - ETA: 18:33 - loss: 1.4600 - accuracy: 0.5273

KeyboardInterrupt: 